### Imports

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model


c:\Users\Max\Documents\Software\5930\finenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load the Dataset

In [2]:
# Load Alpaca dataset from Hugging Face or local JSON
dataset = load_dataset("tatsu-lab/alpaca")

# Preview the dataset structure
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 52002
    })
})


### Format, Scale, and Tokenize the Dataset

In [3]:

# Combine instruction, input, and output into a single text field
def format_example(example):
    instruction = example['instruction']
    input_text = example['input']
    output = example['output']
    if input_text:
        return f"Instruction: {instruction}\nInput: {input_text}\nOutput: {output}"
    else:
        return f"Instruction: {instruction}\nOutput: {output}"

formatted_dataset = dataset.map(lambda x: {"text": format_example(x)})

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-135M")

# Set the padding token to the <eos_token> (end of sentence token)
tokenizer.pad_token = tokenizer.eos_token

# If you want to use a new token for padding
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Tokenize dataset
def tokenize_function(examples):
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()  # Copy input_ids to labels
    return tokenized


tokenized_dataset = formatted_dataset.map(tokenize_function, batched=True)


In [4]:
print(tokenized_dataset)
print(tokenized_dataset["train"][0])




DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 52002
    })
})
{'instruction': 'Give three tips for staying healthy.', 'input': '', 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.', 'text': 'Instruction: Give three tips for staying healthy.\nOutput: 1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.', 'input_ids': [25464, 42, 13843, 1296, 5608, 327, 9286, 2458, 30, 198, 17597, 42, 216, 33, 30, 36693, 253, 8609, 2714, 284, 919, 2090, 288, 1453, 7568, 282, 5574, 284, 5136, 30, 3717, 34, 30, 15382, 5578, 288, 1446, 469, 1248, 3212, 284, 1837, 30, 3717, 35, 30, 5399,

In [5]:
def scale_dataset(dataset, max_samples=1000):
    """Scale down a dataset to a maximum number of samples"""
    if len(dataset) > max_samples:
        scaled_indices = list(range(max_samples))
        return dataset.select(scaled_indices)
    return dataset


# Set your desired size
MAX_SAMPLES = 25000  # Adjust this number as needed

# Print original size
print(f"Original sizes - Total dataset: {len(tokenized_dataset['train'])}")

# First scale down the full dataset
scaled_full = scale_dataset(tokenized_dataset["train"], MAX_SAMPLES)

# Split into train and eval (90-10 split)
splits = scaled_full.train_test_split(
    test_size=0.1,  # 10% for eval
    shuffle=True,
    seed=42
)

tokenized_train = splits['train']
tokenized_eval = splits['test']

# Print final sizes
print(f"Final sizes - Train: {len(tokenized_train)}, Eval: {len(tokenized_eval)}")

Original sizes - Total dataset: 52002
Final sizes - Train: 22500, Eval: 2500


### Fine tuning setup

In [6]:
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from transformers import EarlyStoppingCallback

# Load the pretrained model
model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-135M")

# Configure LoRA
lora_config = LoraConfig(
    r=16,  # Increase from 8 to 16 for more capacity
    lora_alpha=32,  # Keep this the same
    lora_dropout=0.1,  # Increase from 0.05 to 0.1 for better regularization
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],  # Add k_proj and o_proj
    task_type="CAUSAL_LM"
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()


trainable params: 1,843,200 || all params: 136,358,208 || trainable%: 1.3517


In [ ]:
from transformers import TrainingArguments

# training_args = TrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="steps",  # Evaluate periodically
#     eval_steps=5000,              # Evaluate every 5000 steps
#     save_steps=5000,              # Save model every 5000 steps
#     logging_steps=2500,          # Log progress every 2b 500 steps
#     load_best_model_at_end=True,  # Load the best model after training
#     metric_for_best_model="eval_loss",  # Use evaluation loss as the metric
#     greater_is_better=False,     # Lower eval_loss is better
#     learning_rate=5e-4,
#     per_device_train_batch_size=4,
#     num_train_epochs=10,
#     save_total_limit=2,
#     fp16=False,
# )

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=1000,              # Decreased from 5000 to 1000
    save_steps=1000,              # Decreased from 5000 to 1000
    logging_steps=500,            # Decreased from 2500 to 500
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    learning_rate=1e-4,           # Decreased from 5e-4 to 1e-4
    per_device_train_batch_size=8,  # Increased from 4 to 8
    gradient_accumulation_steps=4,   # Added this parameter
    num_train_epochs=3,           # Decreased from 10 to 3
    save_total_limit=2,
    fp16=False,                    # Changed to True
    warmup_ratio=0.03,            # Added warmup
    weight_decay=0.01,            # Added weight decay
)

c:\Users\Max\Documents\Software\5930\finenv\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Train

In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = tokenized_train,
    eval_dataset = tokenized_eval,  # Replace with validation set if available
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

trainer.train()


 24%|██▎       | 500/2109 [08:40<27:51,  1.04s/it]

{'loss': 1.3088, 'grad_norm': 0.04920892417430878, 'learning_rate': 7.867970660146699e-05, 'epoch': 0.71}


 47%|████▋     | 1000/2109 [17:19<19:07,  1.03s/it]

{'loss': 0.288, 'grad_norm': 0.05021047592163086, 'learning_rate': 5.4229828850855746e-05, 'epoch': 1.42}


                                                   
 47%|████▋     | 1000/2109 [17:49<19:07,  1.03s/it]

...

                                                   
 95%|█████████▍| 2000/2109 [35:44<01:53,  1.04s/it]

{'eval_loss': 0.28120020031929016, 'eval_runtime': 30.873, 'eval_samples_per_second': 80.977, 'eval_steps_per_second': 10.138, 'epoch': 2.84}


100%|██████████| 2109/2109 [37:38<00:00,  1.07s/it]

{'train_runtime': 2258.7474, 'train_samples_per_second': 29.884, 'train_steps_per_second': 0.934, 'train_loss': 0.5264985756944174, 'epoch': 3.0}


TrainOutput(global_step=2109, training_loss=0.5264985756944174, metrics={'train_runtime': 2258.7474, 'train_samples_per_second': 29.884, 'train_steps_per_second': 0.934, 'total_flos': 2.239791620161536e+16, 'train_loss': 0.5264985756944174, 'epoch': 2.998933522929257})

In [9]:
# Save the fine-tuned model and tokenizer
output_dir = "./smollm2_finetuned/03"

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model and tokenizer saved to {output_dir}")



Model and tokenizer saved to ./smollm2_finetuned/03
